In [ ]:
from PIL import Image

# Set page config
st.set_page_config(page_title="Finance Agent", layout="wide")

# Custom CSS to remove padding and add custom styling
st.markdown("""
    <style>
    .main .block-container {
        padding-top: 2rem;
        padding-bottom: 2rem;
    }
    .stApp > header {
        background-color: transparent;
    }
    .stApp {
        margin: 0 auto;
    }
    </style>
""", unsafe_allow_html=True)

# Context managers for redirecting stdout/stderr (unchanged)
@contextmanager
def st_redirect(src, dst):
    placeholder = st.empty()
    output_func = getattr(placeholder, dst)

    with StringIO() as buffer:
        old_write = src.write

        def new_write(b):
            if add_script_run_ctx():
                buffer.write(b)
                output_func(buffer.getvalue())
            else:
                old_write(b)

        try:
            src.write = new_write
            yield
        finally:
            src.write = old_write

@contextmanager
def st_stdout(dst):
    with st_redirect(sys.stdout, dst):
        yield

@contextmanager
def st_stderr(dst):
    with st_redirect(sys.stderr, dst):
        yield

# Create a custom-width container
_, center_column, _ = st.columns([1, 3, 1])

with center_column:
    st.title("Finance Agent Query Interface")

    # Create two columns: left for main content, right for logs
    left_column, right_column = st.columns([2, 1])

    # Left column: Design diagram and query input
    with left_column:
        # Display design diagram
        st.subheader("Design Diagram")
        image = Image.open("assets/design_diagram.png")
        st.image(image, use_column_width=True)

        # Query input
        st.subheader("Query Input")
        query = st.text_area("Enter your finance-related query:", height=100)
        run_button = st.button("Run Query", type="primary")

    # Right column: Process Log
    with right_column:
        st.subheader("Process Log")
        log_container = st.empty()

    # Main content area for results (below query in left column)
    result_container = left_column.container()

    if run_button and query:
        with log_container:
            with st_stdout("code"):
                result = asyncio.run(run_finance_agent(query))

        with result_container:
            st.success("Query processing completed!")
            
            # Display graph database results if available
            if 'graph_db_result' in result:
                st.subheader("Graph Database Results")
                st.json(result['graph_db_result'])
            
            # Display expanded queries if available
            if 'expanded_queries' in result:
                st.subheader("Expanded Queries")
                for i, eq in enumerate(result['expanded_queries'], 1):
                    st.write(f"{i}. {eq}")
            
            # Display subgraph results if available
            if 'subgraph_results' in result:
                st.subheader("Subgraph Results")
                for subgraph, subresult in result['subgraph_results'].items():
                    with st.expander(f"Results from {subgraph}"):
                        st.json(subresult)
            
            # Display final response
            st.subheader("Final Response")
            st.write(result.get('final_response', 'No final response available.'))

    else:
        result_container.info("Enter a query and click 'Run Query' to start processing.")

    # Footer
    st.markdown("---")
    st.markdown("© 2023 Finance Agent. All rights reserved.")

In [ ]:
graph TD
    A[Initial Query] --> B{Graph DB Router}
    B -->|Use Graph DB| C[Query Graph DB]
    C --> D[Expand Queries]
    D --> E[Subgraph Router]
    B -->|Skip Graph DB| F[Refine Query]
    F --> E
    
    subgraph ECT Subgraph
        G[Parse ECT Schema]
        R{Schema Valid?}
        H[Perform ECT Search]
        I[Generate ECT Response]
        G --> R
        R -->|Yes| H
        R -->|No, Retry| G
        H --> I
    end
    
    E -.-> G
    I --> J[Result Aggregation]
    
    E -.-> K[Other Subgraphs]
    K --> J
    
    J --> L[Final Response Generation]
    
    style A fill:#f9f,stroke:#333,stroke-width:2px
    style B fill:#bbf,stroke:#f66,stroke-width:2px,color:#fff
    style C fill:#bfe,stroke:#333,stroke-width:2px
    style D fill:#bfe,stroke:#333,stroke-width:2px
    style E fill:#fbb,stroke:#666,stroke-width:2px
    style F fill:#feb,stroke:#333,stroke-width:2px
    style G fill:#e0e0e0,stroke:#999,stroke-width:2px
    style R fill:#ffcc00,stroke:#ff9900,stroke-width:2px
    style H fill:#e0e0e0,stroke:#999,stroke-width:2px
    style I fill:#e0e0e0,stroke:#999,stroke-width:2px
    style J fill:#bfb,stroke:#333,stroke-width:2px
    style K fill:#e0e0e0,stroke:#999,stroke-width:2px
    style L fill:#f9f,stroke:#333,stroke-width:2px
    
    %% Edge styles
    linkStyle 1 stroke:#0f0,stroke-width:2px;
    linkStyle 2 stroke:#0f0,stroke-width:2px;
    linkStyle 3 stroke:#0f0,stroke-width:2px;
    linkStyle 4 stroke:#f90,stroke-width:2px;
    linkStyle 5 stroke:#f90,stroke-width:2px;
    linkStyle 10 stroke:#666,stroke-width:2px,stroke-dasharray: 5 5;
    linkStyle 12 stroke:#666,stroke-width:2px,stroke-dasharray: 5 5;